# Instalar dependências

In [18]:
# """
# Instalação de bibliotecas necessárias para o projeto.
# Inclui YOLO, manipulação de dados e visualização.
# """
!pip3 install ultralytics kagglehub pandas scikit-learn matplotlib seaborn opencv-python


^C



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Instalar CUDA (PyTorch com GPU) 

In [13]:
"""
Instalação do PyTorch com suporte a CUDA 12.6.
Permite treinamento acelerado por GPU.
"""
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

# Verificar instalação CUDA
import torch
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"CUDA versão: {torch.version.cuda}")
else:
    print("AVISO: GPU não detectada. Treinamento usará CPU.")


Looking in indexes: https://download.pytorch.org/whl/cu126
CUDA disponível: True
GPU: NVIDIA GeForce RTX 2050
Memória GPU: 4.00 GB
CUDA versão: 12.6



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Baixar dataset do Kaggle


In [ ]:
# """
# Download e organização do dataset MUID-IITR.
# """
# import kagglehub
# import shutil
# import os

# print("Baixando dataset...")
# path = kagglehub.dataset_download("lakshyataragi/mobilephoneusagedatasetiitr")
# print(f"Dataset baixado em: {path}")

# # Mover para diretório de trabalho
# pasta_atual = os.getcwd()
# for arquivo in os.listdir(path):
#     origem = os.path.join(path, arquivo)
#     destino = os.path.join(pasta_atual, arquivo)
    
#     if os.path.exists(destino):
#         if os.path.isdir(destino):
#             shutil.rmtree(destino)
#         else:
#             os.remove(destino)
    
#     shutil.move(origem, destino)
#     print(f"Movido: {arquivo}")

# # Limpar cache
# cache_path = os.path.expanduser("~/.cache/kagglehub")
# if os.path.exists(cache_path):
#     shutil.rmtree(cache_path)
#     print("\nCache limpo")

# print("\nArquivos disponíveis:")
# for item in sorted(os.listdir(pasta_atual)):
#     if os.path.isdir(item) and item in ['positive', 'negative']:
#         count = len([f for f in os.listdir(item) if f.endswith(('.jpg', '.png', '.jpeg'))])
#         print(f"  [DIR]  {item:15s} ({count} imagens)")
#     elif item.endswith('.csv'):
#         print(f"  [FILE] {item}")

# Preparar dataset para YOLO

In [ ]:
"""
Preparação do dataset com verificações robustas.
Formato YOLO com split estratificado.
"""
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import os

def preparar_dataset():
    """Prepara dataset no formato YOLO otimizado para treinamento."""
    
    # Criar estrutura de diretórios
    diretorios = [
        'dataset/images/train',
        'dataset/images/val',
        'dataset/labels/train',
        'dataset/labels/val'
    ]
    
    for dir_path in diretorios:
        os.makedirs(dir_path, exist_ok=True)
    
    # Coletar imagens positive
    positive_imgs = []
    if os.path.exists('positive'):
        for img in os.listdir('positive'):
            if img.lower().endswith(('.jpg', '.png', '.jpeg')):
                positive_imgs.append(('positive/' + img, 1))
    
    # Coletar imagens negative
    negative_imgs = []
    if os.path.exists('negative'):
        for img in os.listdir('negative'):
            if img.lower().endswith(('.jpg', '.png', '.jpeg')):
                negative_imgs.append(('negative/' + img, 0))
    
    # Validações
    total_positive = len(positive_imgs)
    total_negative = len(negative_imgs)
    
    print(f"Imagens positive: {total_positive}")
    print(f"Imagens negative: {total_negative}")
    print(f"Total:            {total_positive + total_negative}")
    
    if total_positive == 0:
        raise ValueError("ERRO: Nenhuma imagem positive encontrada")
    
    if total_negative == 0:
        print("AVISO: Nenhuma imagem negative encontrada")
        print("Modelo treinará apenas com positives")
    
    # Split estratificado (80% treino, 20% validação)
    all_imgs = positive_imgs + negative_imgs
    train_imgs, val_imgs = train_test_split(
        all_imgs,
        test_size=0.2,
        random_state=42,
        stratify=[label for _, label in all_imgs] if total_negative > 0 else None
    )
    
    # Processar conjunto de treino
    for img_path, label in train_imgs:
        img_name = os.path.basename(img_path)
        dest_path = f'dataset/images/train/{img_name}'
        shutil.copy2(img_path, dest_path)
        
        if label == 1:
            label_name = os.path.splitext(img_name)[0] + '.txt'
            label_file = f'dataset/labels/train/{label_name}'
            with open(label_file, 'w') as f:
                # Formato YOLO: classe x_center y_center width height (0-1)
                f.write('0 0.5 0.5 1.0 1.0\n')
    
    # Processar conjunto de validação
    for img_path, label in val_imgs:
        img_name = os.path.basename(img_path)
        dest_path = f'dataset/images/val/{img_name}'
        shutil.copy2(img_path, dest_path)
        
        if label == 1:
            label_name = os.path.splitext(img_name)[0] + '.txt'
            label_file = f'dataset/labels/val/{label_name}'
            with open(label_file, 'w') as f:
                f.write('0 0.5 0.5 1.0 1.0\n')
    
    # Estatísticas
    train_positive = sum(1 for _, label in train_imgs if label == 1)
    val_positive = sum(1 for _, label in val_imgs if label == 1)
    
    print(f"\nDataset preparado com sucesso:")
    print(f"  Treino:     {len(train_imgs)} imagens ({train_positive} positive)")
    print(f"  Validação:  {len(val_imgs)} imagens ({val_positive} positive)")
    print(f"  Proporção:  {len(train_imgs)/len(all_imgs)*100:.1f}% treino / {len(val_imgs)/len(all_imgs)*100:.1f}% validação")
    
    return len(train_imgs), len(val_imgs)

# Executar preparação
train_count, val_count = preparar_dataset()

✓ Treino: 710 | Validação: 178


# Criar arquivo de configuração YOLO

In [ ]:
"""
Arquivo de configuração do dataset.
"""
config_content = """path: ./dataset
train: images/train
val: images/val

nc: 1
names: ['usando_celular']
"""

with open('dataset.yaml', 'w') as f:
    f.write(config_content)

print("Arquivo dataset.yaml criado")

Arquivo dataset.yaml criado com sucesso.


# Treinar modelo YOLOv8

In [ ]:
"""
Treinamento otimizado para Linux com RTX 2050 4GB.
Configurações ajustadas.
"""
from ultralytics import YOLO
import torch
import os

# ===== CONFIGURAÇÕES =====
TRAIN_GPU = 1
EPOCHS = 100                    # Mais epochs para melhor resultado
BATCH_SIZE = 12                 # Otimizado para 4GB em Linux
IMG_SIZE = 640
PATIENCE = 20
MODELO_BASE = 'yolov8m.pt'      # Modelo medium para melhor precisão

# Verificar ambiente
print("="*60)
print("CONFIGURAÇÃO DE TREINAMENTO")
print("="*60)

cuda_available = torch.cuda.is_available()
print(f"Sistema:             Linux")
print(f"PyTorch:             {torch.__version__}")
print(f"CUDA disponível:     {cuda_available}")

if cuda_available:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print(f"GPU:                 {gpu_name}")
    print(f"Memória GPU:         {gpu_mem:.2f} GB")
    print(f"CUDA versão:         {torch.version.cuda}")
    print(f"cuDNN:               {torch.backends.cudnn.version()}")
    
    # Otimizações CUDA
    torch.backends.cudnn.benchmark = True      # Auto-tuning
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    device = 0
    print("\nOtimizações CUDA:    ATIVADAS")
    print("  - cuDNN benchmark: True")
    print("  - TF32:            True")
    
    # Ajustar batch baseado na memória e modelo
    if gpu_mem < 6:
        if 'm' in MODELO_BASE:
            BATCH_SIZE = 12
        elif 'l' in MODELO_BASE or 'x' in MODELO_BASE:
            BATCH_SIZE = 8
        else:
            BATCH_SIZE = 16
        print(f"\nBatch ajustado:      {BATCH_SIZE}")
else:
    device = 'cpu'
    print("\nERRO: GPU não detectada!")
    print("Execute: nvidia-smi")
    raise RuntimeError("GPU NVIDIA não encontrada")

# Carregar modelo
print(f"\nCarregando {MODELO_BASE}...")
try:
    model = YOLO(MODELO_BASE)
    print(f"Modelo carregado: {MODELO_BASE}")
except Exception as e:
    print(f"ERRO: {e}")
    raise

# Configuração final
print(f"\n{'='*60}")
print("PARÂMETROS DE TREINAMENTO")
print(f"{'='*60}")
print(f"Modelo:              {MODELO_BASE}")
print(f"Epochs:              {EPOCHS}")
print(f"Batch Size:          {BATCH_SIZE}")
print(f"Image Size:          {IMG_SIZE}x{IMG_SIZE}")
print(f"Dispositivo:         GPU {device}")
print(f"Workers:             8 (Linux multiprocessing)")
print(f"Patience:            {PATIENCE}")
print(f"AMP:                 Habilitado")
print(f"{'='*60}\n")

# Treinar com configurações otimizadas para Linux
try:
    results = model.train(
        data='dataset.yaml',
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        patience=PATIENCE,
        device=device,
        name='celular_detector',
        
        # Performance
        workers=8,                      # Linux permite multiprocessing
        amp=True,                       # Automatic Mixed Precision
        
        # Otimizações
        cache=True,                     # Cache imagens em RAM
        optimizer='AdamW',              # Otimizador mais eficiente
        cos_lr=True,                    # Cosine learning rate
        
        # Augmentations otimizadas
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=10.0,
        translate=0.1,
        scale=0.5,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.1,
        
        # Configurações
        verbose=True,
        exist_ok=True,
        plots=True,
        save=True,
        val=True,
        save_period=10,                 # Salvar checkpoint a cada 10 epochs
        
        # Callbacks
        patience=PATIENCE,
        close_mosaic=int(EPOCHS * 0.8)  # Desabilitar mosaic nos últimos 20%
    )
    
    print("\n" + "="*60)
    print("TREINAMENTO CONCLUÍDO")
    print("="*60)
    print(f"Modelo salvo em: runs/detect/celular_detector/weights/best.pt")
    print(f"Logs em:         runs/detect/celular_detector/")
    
    # Avaliar modelo
    print("\nAvaliando modelo...")
    metrics = model.val(
        data='dataset.yaml',
        workers=8,
        batch=BATCH_SIZE
    )
    
    # Exibir métricas
    print(f"\n{'='*60}")
    print("MÉTRICAS DE DESEMPENHO")
    print(f"{'='*60}")
    print(f"Modelo:              {MODELO_BASE}")
    print(f"mAP50:               {metrics.box.map50:.4f}")
    print(f"mAP50-95:            {metrics.box.map:.4f}")
    print(f"Precision:           {metrics.box.mp:.4f}")
    print(f"Recall:              {metrics.box.mr:.4f}")
    
    if (metrics.box.mp + metrics.box.mr) > 0:
        f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr)
        print(f"F1-Score:            {f1:.4f}")
    
    print(f"{'='*60}")
    
    # Avaliação qualitativa
    if metrics.box.map50 > 0.85:
        print("\nRESULTADO: Excelente (>85%)")
    elif metrics.box.map50 > 0.70:
        print("\nRESULTADO: Bom (70-85%)")
    elif metrics.box.map50 > 0.50:
        print("\nRESULTADO: Aceitável (50-70%)")
    else:
        print("\nRESULTADO: Baixo (<50%) - Revisar dataset")
    
    # Limpar cache GPU
    torch.cuda.empty_cache()
    print("\nCache GPU limpo")

except RuntimeError as e:
    error_msg = str(e).lower()
    
    if "out of memory" in error_msg:
        print("\nERRO: Memória GPU insuficiente")
        print("\nREDUZIR:")
        print(f"  BATCH_SIZE = {BATCH_SIZE // 2}")
        print(f"  IMG_SIZE = 416")
        print(f"  cache=False")
    else:
        print(f"\nERRO: {e}")
    
    torch.cuda.empty_cache()
    raise

except KeyboardInterrupt:
    print("\n\nTreinamento interrompido pelo usuário")
    torch.cuda.empty_cache()

except Exception as e:
    print(f"\nERRO inesperado: {e}")
    torch.cuda.empty_cache()
    raise

Verificando ambiente de treinamento...

PyTorch versão: 2.9.0+cu126
CUDA disponível: True
GPU: NVIDIA GeForce RTX 2050
Memória GPU: 4.00 GB
CUDA versão: 12.6

Modo: GPU ATIVADA

Carregando yolov8s.pt...
Aguarde o download se for primeira execução...
Modelo yolov8s.pt carregado com sucesso.

Iniciando treinamento:
  Modelo:       yolov8s.pt
  Epochs:       50
  Batch Size:   8
  Img Size:     640x640
  Dispositivo:  0
  Patience:     15
------------------------------------------------------------
New https://pypi.org/project/ultralytics/8.3.221 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.220  Python-3.12.0 torch-2.9.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, dete

KeyboardInterrupt: 

# Testar modelo em imagens

In [ ]:
"""
Teste do modelo treinado com visualização.
"""
from ultralytics import YOLO
import cv2
from IPython.display import Image, display
import os

# Carregar melhor modelo
model = YOLO('runs/detect/celular_detector/weights/best.pt')

# Testar em imagens
test_images = []
if os.path.exists('positive'):
    test_images = [f'positive/{img}' for img in os.listdir('positive')[:5]
                   if img.lower().endswith(('.jpg', '.png', '.jpeg'))]

print(f"Testando em {len(test_images)} imagens...\n")

for img_path in test_images:
    results = model(img_path, conf=0.5, verbose=False)
    
    output_name = f"resultado_{os.path.basename(img_path)}"
    results[0].save(output_name)
    
    num_det = len(results[0].boxes)
    print(f"{os.path.basename(img_path)}: {num_det} detecções")
    
    if num_det > 0:
        conf = results[0].boxes.conf[0].item()
        print(f"  Confiança: {conf:.2%}")
    
    display(Image(output_name))

# Métricas avançadas

In [ ]:
"""
Análise detalhada com matriz de confusão e curvas.
"""
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

model = YOLO('runs/detect/celular_detector/weights/best.pt')

print("Calculando métricas avançadas...\n")

# Coletar predições
val_images = [os.path.join('dataset/images/val', img)
              for img in os.listdir('dataset/images/val')]

y_true = []
y_pred = []

print(f"Processando {len(val_images)} imagens...")
for i, img_path in enumerate(val_images):
    if (i + 1) % 20 == 0:
        print(f"  {i+1}/{len(val_images)}")
    
    # Ground truth
    img_name = os.path.basename(img_path)
    label_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join('dataset/labels/val', label_name)
    
    has_label = os.path.exists(label_path) and os.path.getsize(label_path) > 0
    y_true.append(1 if has_label else 0)
    
    # Predição
    results = model(img_path, conf=0.5, verbose=False)
    y_pred.append(1 if len(results[0].boxes) > 0 else 0)

print(f"Total: {len(val_images)}\n")

# Matriz de Confusão
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Sem Celular', 'Com Celular'],
            yticklabels=['Sem Celular', 'Com Celular'],
            annot_kws={'size': 14},
            cbar_kws={'label': 'Quantidade'})
plt.title('Matriz de Confusão - Detecção de Uso de Celular',
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Valor Real', fontsize=13)
plt.xlabel('Predição do Modelo', fontsize=13)
plt.tight_layout()
plt.savefig('matriz_confusao.png', dpi=200, bbox_inches='tight')
plt.show()

print("="*60)
print("MATRIZ DE CONFUSÃO")
print("="*60)
print(f"Verdadeiros Negativos (TN):  {cm[0,0]:5d}  (Correto: SEM)")
print(f"Falsos Positivos (FP):       {cm[0,1]:5d}  (Erro: Falso alarme)")
print(f"Falsos Negativos (FN):       {cm[1,0]:5d}  (Erro: Não detectou)")
print(f"Verdadeiros Positivos (TP):  {cm[1,1]:5d}  (Correto: COM)")

# Curva Precision-Recall
thresholds = np.arange(0.1, 1.0, 0.05)
precisions = []
recalls = []

print(f"\nCalculando curva P-R ({len(thresholds)} pontos)...")
for thresh in thresholds:
    y_pred_t = []
    for img_path in val_images:
        results = model(img_path, conf=thresh, verbose=False)
        y_pred_t.append(1 if len(results[0].boxes) > 0 else 0)
    
    tp = sum((yt == 1 and yp == 1) for yt, yp in zip(y_true, y_pred_t))
    fp = sum((yt == 0 and yp == 1) for yt, yp in zip(y_true, y_pred_t))
    fn = sum((yt == 1 and yp == 0) for yt, yp in zip(y_true, y_pred_t))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    precisions.append(precision)
    recalls.append(recall)

plt.figure(figsize=(12, 7))
plt.plot(recalls, precisions, 'b-', linewidth=3, marker='o', markersize=5)
plt.fill_between(recalls, precisions, alpha=0.2)
plt.xlabel('Recall (Sensibilidade)', fontsize=13)
plt.ylabel('Precision (Precisão)', fontsize=13)
plt.title('Curva Precision-Recall', fontsize=16, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3, linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])

for i, thresh in enumerate(thresholds[::3]):
    idx = i * 3
    if idx < len(recalls):
        plt.annotate(f'{thresh:.2f}',
                    xy=(recalls[idx], precisions[idx]),
                    xytext=(8, 8), textcoords='offset points',
                    fontsize=9, alpha=0.7,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

plt.tight_layout()
plt.savefig('precision_recall_curve.png', dpi=200, bbox_inches='tight')
plt.show()

# Relatório
print("\n" + "="*60)
print("RELATÓRIO DE CLASSIFICAÇÃO")
print("="*60)
report = classification_report(y_true, y_pred,
                               target_names=['Sem Celular', 'Com Celular'],
                               digits=4)
print(report)

# Resumo
accuracy = (cm[0,0] + cm[1,1]) / cm.sum() if cm.sum() > 0 else 0
specificity = cm[0,0] / (cm[0,0] + cm[0,1]) if (cm[0,0] + cm[0,1]) > 0 else 0
sensitivity = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0

print("="*60)
print("RESUMO FINAL")
print("="*60)
print(f"Acurácia (Accuracy):     {accuracy:.2%}")
print(f"Especificidade:          {specificity:.2%}  (Taxa de TN)")
print(f"Sensibilidade (Recall):  {sensitivity:.2%}  (Taxa de TP)")
print(f"Total Imagens:           {len(y_true)}")
print(f"\nGráficos salvos:")
print(f"  - matriz_confusao.png")
print(f"  - precision_recall_curve.png")
print("="*60)